---   
 <img align="left" width="75" height="75"  src="https://upload.wikimedia.org/wikipedia/en/c/c8/University_of_the_Punjab_logo.png"> 

<h1 align="center">Department of Data Science</h1>
<h1 align="center">Course: Tools and Techniques for Data Science</h1>

---
<h3><div align="right">Instructor: Muhammad Arif Butt, Ph.D.</div></h3>    

<h1 align="center">Lecture 3.16 (Pandas-08)</h1>

<img align="right" width="400" height="400"  src="images/pandas-apps.png"  >

## _Handling Missing Data.ipynb_

## Learning agenda of this notebook

1. Have an insight about the Dataset
2. Identify the Columns having Null/Missing values using `df.isna()` method
3. Handle/Impute the Null/Missing Values under the `math` Column using `df.loc[mask,col]=value`
4. Handle/Impute the Null/Missing Values under the `group` Column using `df.loc[mask,col]=value`
5. Handle Missing values under a Numeric/Categorical Column using `fillna()`
6. Handle Repeating Values (for same information) under the `session` Column
7. Create a new Column by Modifying an Existing Column
8. Delete Rows Having NaN values using `df.dropna()` method
9. Convert Categorical Variables into Numerical

## 1. Have an Insight about the Dataset

In [1]:
! cat datasets/group-marks.csv



















































MS50,KAKAMANNA,male,group C,AFTERNOON,37,3000,66,78,81

In [2]:
# import the pandas library
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv')
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [3]:
df.shape

(50, 10)

- Whenever the **`pd.read.csv()`** method detects a missing value (nothing between two commas in a csv file or an empty cell in Excel) it flags it with NaN. There can be many reasons for these NaN values, one can be that the data is gathered via google form from people and this field might be optional and skipped.
- There can also be a scenario that a user has entered some text under a numeric field about which he/she do not have any information.

## 2. Identify the Columns having Null/Missing values
- The **`df.isna()`** method isrecommended to use than `df.isnull()`, which return a boolean same-sized object that indicates whether an element is NA value or not. Missing values get mapped to True. Everything else gets mapped to False values. Remember, characters such as empty strings ``''`` or `numpy.inf` are not considered NA values.
- The **`df.notna()`** method is recommended to use than `df.notnull()` methods return a boolean same-sized object that indicates whether an element is NA value or not. Non-missing values get mapped to True. 

In [4]:
df.isna().head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [5]:
df.notna().head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True
2,True,True,True,False,True,True,True,False,True,True
3,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True


In [6]:
# Now we can use sum() on this dataframe object of Boolean values (True is mapped to 1)
df.isna().sum()

rollno         0
name           0
gender         0
group          3
session        0
age            0
scholarship    0
math           4
english        3
urdu           0
dtype: int64

In [7]:
# Similarly, we can use sum() on this dataframe object of Boolean values (True is is mapped to 1)
df.notna().sum()

rollno         50
name           50
gender         50
group          47
session        50
age            50
scholarship    50
math           46
english        47
urdu           50
dtype: int64

## 3. Handle/Impute the Null/Missing Values under the `math` Column

### a. Identify the Rows under the `math` Column having Null/Missing values
- The `df.isna()` method works equally good on Series objects as well

In [9]:
mask = df.math.isna()
mask

0     False
1     False
2      True
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23     True
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44     True
45    False
46    False
47    False
48    False
49    False
Name: math, dtype: bool

In [10]:
# This will return only those rows of dataframe having null values under the math column
df[mask]         # df[df.math.isna()]
df.loc[mask, :]  # df.loc[df.math.isna(), :]

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
5,MS06,SAFIA,female,group B,AFT,23,3800,NaN,83.0,78
23,MS24,LAIBA,female,group C,AFTERNOON,37,3000,NaN,73.0,73
44,MS45,ZAINAB,female,group E,MOR,28,3500,NaN,56.0,54


### b. Replace the Null/Missing Values under the `math` Column
- After detecting the NaN values, the next question is, what value we should write in the cells where we have Null/Missing values under the `math` column
- Suppose, we want to put the average values at the place of missing values.

In [11]:
# Compute the mean of math column
df.math.mean() 

TypeError: can only concatenate str (not "int") to str

> By seeing the error, it appears that the `math` column do not have the `int64` or `float64` type. Let us check this out

In [12]:
# Check out the data type of math column
df['math'].dtypes

dtype('O')

In [13]:
# We can also use the `df.info()` method to display the count of Non-Null columns, their datatypes, their names 
# and memory usage of that dataframe.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rollno       50 non-null     object 
 1   name         50 non-null     object 
 2   gender       50 non-null     object 
 3   group        47 non-null     object 
 4   session      50 non-null     object 
 5   age          50 non-null     int64  
 6   scholarship  50 non-null     int64  
 7   math         46 non-null     object 
 8   english      47 non-null     float64
 9   urdu         50 non-null     int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.0+ KB


- **What can be the reason for this?**
- Let us check out the values under this column

In [14]:
df['math']

0     No Idea
1          69
2         NaN
3          47
4          76
5         NaN
6          88
7          40
8          64
9          38
10         58
11         40
12         65
13         78
14         50
15         69
16         88
17         18
18         46
19         54
20         66
21         65
22         44
23        NaN
24         74
25         73
26         69
27         67
28         70
29         62
30         69
31         63
32         56
33         40
34         97
35         81
36         74
37         50
38         75
39         57
40         55
41         58
42         53
43         59
44        NaN
45         65
46         55
47         66
48         57
49         66
Name: math, dtype: object

In [15]:
# We can replace all such values using the `replace()` method
import numpy as np
df.replace('No Idea', np.nan).head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [16]:
# Note the marks of Saadia in math are changed from string `No Idea` to `NaN`
# Since this seems working fine let us make inplace=True to make these changes in the original dataframe
df.replace('No Idea', np.nan, inplace=True)

In [18]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [17]:
# Let us check the data type of math column
df['math'].dtypes

dtype('O')

In [19]:
# It is still Object, which is natural, however, we can change the datatype to `df.astype()` method
df['math'] = df['math'].astype(float)

In [20]:
# Let us check the data type of math column
df['math'].dtypes

dtype('float64')

In [21]:
# Let us compute the average of math marks again 
df.math.mean() 

61.644444444444446

In [22]:
# List only those records under math column having Null values
mask = df.math.isna()
df.loc[mask, 'math']

0    NaN
2    NaN
5    NaN
23   NaN
44   NaN
Name: math, dtype: float64

In [23]:
# Let us replace these values with mean value of the math column
df.loc[(df.math.isna()),'math'] = df.math.mean()

In [24]:
# Confirm the result
df.isna().sum()
#df.info()

rollno         0
name           0
gender         0
group          3
session        0
age            0
scholarship    0
math           0
english        3
urdu           0
dtype: int64

In [25]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,61.644444,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.000000,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,61.644444,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.000000,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.000000,78.0,55


## 4. Handle/Impute the Null/Missing Values under the `group` Column
- The `group` column contains categorical values, i.e., a value that can take on one of a limited, and usually fixed, number of possible values.

### a. Identify the Rows under the `group` Column having Null/Missing values

In [26]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,61.644444,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.000000,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,61.644444,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.000000,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.000000,78.0,55


In [27]:
mask = df.group.isna()
mask.head()

0    False
1    False
2     True
3    False
4    False
Name: group, dtype: bool

In [28]:
df[mask]          # df[df.group.isna()]
df.loc[mask, :]   # df.loc[df.group.isna()]

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
2,MS03,ARIFA,female,NaN,EVENING,34,3500,61.644444,95.0,93
12,MS13,MAHOOR,female,NaN,MOR,25,2345,65.000000,81.0,73
32,MS33,SHAISTA,female,NaN,MORNING,29,3500,56.000000,72.0,65


### b. Replace the Null/Missing Values under the `group` Column
- After detecting the NaN values, the next question is, what value we should write in the cells where we have Null/Missing values
- Since this is a categorical column having datatype object (group A, group B, group C, ...), so let us replace it with th value inside the column having the maximum frequency

In [29]:
# Use value_counts() function which return a Series containing counts of unique values (in descending order)
# with the most frequently-occurring element at first. It excludes NA values by default.
df.group.value_counts()

group C    14
group B    13
group D    12
group A     5
group E     3
Name: group, dtype: int64

In [30]:
# Another way of doing is use the mode() function on the column
df.group.mode() 

0    group C
dtype: object

In [31]:
# List only those records under group column having Null values
mask = df.group.isna()
df.loc[mask, 'group']     # df.loc[(df.group.isna()), 'group']

2     NaN
12    NaN
32    NaN
Name: group, dtype: object

In [32]:
# Let us replace these values with maximum occurring value in the `group` column
df.loc[(df.group.isna()),'group'] = 'group C'

In [33]:
# Confirm the result
df.isna().sum()
#df.info()

rollno         0
name           0
gender         0
group          0
session        0
age            0
scholarship    0
math           0
english        3
urdu           0
dtype: int64

In [34]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,61.644444,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.000000,90.0,88
2,MS03,ARIFA,female,group C,EVENING,34,3500,61.644444,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.000000,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.000000,78.0,55


>Note that in the original dataframe Arifa group information was missing, and now it is `group C` 

## 5. Handle Missing values under a Numeric/Categorical Column using `fillna()`

### a. Replace the Null/Missing Values under the math Column using `fillna()`
- This is more recommended way of filling in the Null values within columns of your dataset rather than the use of the `loc` method.
```
object.fillna(value, method, inplace=True)
```
- The only required argument is either the `value`, with which we want to replace the missing values OR the `method` to be used to replace the missing values
- Returns object with missing values filled or None if ``inplace=True``

In [35]:
# Let us read the dataset again with NA values under math column
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv')

In [36]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


>- Before proceeding, let us this time handle the string value `No Idea` under the math column while reading the csv file, instead of doing afterwards in the dataframe using the `replace()` method as we have done above.
>- For this we will use the `na_values` argument to the `pd.read_csv()` method, to which you can pass a single value or a list of values to be replaced with NaN

In [37]:
df = pd.read_csv('datasets/group-marks.csv', na_values='No Idea')

In [38]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.0,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.0,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.0,78.0,55


In [39]:
df.isna().sum()

rollno         0
name           0
gender         0
group          3
session        0
age            0
scholarship    0
math           5
english        3
urdu           0
dtype: int64

In [40]:
df.loc[df.math.isna()]

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
5,MS06,SAFIA,female,group B,AFT,23,3800,NaN,83.0,78
23,MS24,LAIBA,female,group C,AFTERNOON,37,3000,NaN,73.0,73
44,MS45,ZAINAB,female,group E,MOR,28,3500,NaN,56.0,54


In [41]:
# This time instead of loc, use fillna() method with just two arguments
# inplace=True parameter ensure that this happens in the original dataframe

df.math.fillna(value=df.math.mean(), inplace=True)

In [42]:
# Confirm the result
df.isna().sum()
#df.info()

rollno         0
name           0
gender         0
group          3
session        0
age            0
scholarship    0
math           0
english        3
urdu           0
dtype: int64

In [43]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,61.644444,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.000000,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,61.644444,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.000000,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.000000,78.0,55


### b. Replace the Null/Missing Values under the `group` Column using `fillna()`

In [44]:
# Let us read the dataset again with NA values
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv', na_values='No Idea')
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.0,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.0,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.0,78.0,55


In [45]:
df.isna().sum()

rollno         0
name           0
gender         0
group          3
session        0
age            0
scholarship    0
math           5
english        3
urdu           0
dtype: int64

In [46]:
# Once again instead of loc,let us use fillna() method with just two arguments

df.group.fillna('group C', inplace=True)

In [47]:
# Confirm the result
df.isna().sum()
#df.info()

rollno         0
name           0
gender         0
group          0
session        0
age            0
scholarship    0
math           5
english        3
urdu           0
dtype: int64

In [48]:
# Let us fill the math, english and scholarship columns as well again
df.math.fillna(df.math.mean(), inplace=True)
df.english.fillna(df.english.mean(), inplace=True)
df.scholarship.fillna(df.scholarship.mean(), inplace=True)

In [49]:
# Confirm the result
df.isna().sum()


rollno         0
name           0
gender         0
group          0
session        0
age            0
scholarship    0
math           0
english        0
urdu           0
dtype: int64

### c. Replace the Null/Missing Values under the` math` and `group` Column using `ffill` and `bfill` Arguments
- In above examples, we have used the mean value in case of numeric column and mode value in case of a categorical column as the filling value to the `fillna()` method
```
object.fillna(value, method, inplace=True)
```

- We can pass `ffill` or `bfill` as method argument to the `ffillna()` method. This will replace the null values with other values from the DataFrame
- `ffill` (Forward fill): It fills the NaN value with the previous value
- `bfill` (Back fill): It fills the NaN value with the Next/Upcoming value

<img align="right" width="490" height="100"  src="images/bfill.png"  >
<img align="left" width="490" height="100"  src="images/ffill.png"  >

In [50]:
# Let us read the dataset again with NA values
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv', na_values='No Idea')
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.0,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.0,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.0,78.0,55


In [51]:
df.isna().sum()

rollno         0
name           0
gender         0
group          3
session        0
age            0
scholarship    0
math           5
english        3
urdu           0
dtype: int64

In [52]:
# forward fill or ffill attribute
# If have NaN value, just carry forward the previous value
# using ffill attribute, you can fill the NaN value with the previous value in that column
df.fillna(method = 'ffill', inplace=True)
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,NaN,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.0,90.0,88
2,MS03,ARIFA,female,group C,EVENING,34,3500,69.0,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.0,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.0,78.0,55


>Is it working fine?

In [53]:
df.fillna(method = 'bfill', inplace=True)
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,69.0,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69.0,90.0,88
2,MS03,ARIFA,female,group C,EVENING,34,3500,69.0,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47.0,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76.0,78.0,55


In [54]:
# Confirm the result
df.isna().sum()

rollno         0
name           0
gender         0
group          0
session        0
age            0
scholarship    0
math           0
english        0
urdu           0
dtype: int64

## 6. Handle Repeating Values (for same information) under the `session` Column
- If you observe the values under the `session` column, you can observe that it is a categorical column containing six different categories (as values).
    - Notice that the categories `MORNING` and `MOR` are same
    - Similarly, `AFTERNOON` and `AFT` are same
    - Similarly, `EVENING` and `EVE` are same
- This happens when you have collected data from different sources, where same information is written in different ways
- So the `session` column has six different categories (as values) but should have only three

In [56]:
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv' )
df

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55
5,MS06,SAFIA,female,group B,AFT,23,3800,NaN,83.0,78
6,MS07,SARA,female,group B,EVENING,47,3000,88,95.0,92
7,MS08,ABDULLAH,male,group B,EVE,33,2000,40,43.0,39
8,MS09,KHAN,male,group D,MORNING,27,2500,64,NaN,67
9,MS10,HASEENA,female,group B,AFT,33,2800,38,60.0,50


In [57]:
df.session

0       MORNING
1     AFTERNOON
2       EVENING
3           MOR
4     AFTERNOON
5           AFT
6       EVENING
7           EVE
8       MORNING
9           AFT
10          MOR
11      MORNING
12          MOR
13    AFTERNOON
14          AFT
15      EVENING
16          MOR
17    AFTERNOON
18          AFT
19      MORNING
20    AFTERNOON
21      EVENING
22          MOR
23    AFTERNOON
24          AFT
25      EVENING
26          EVE
27      MORNING
28          AFT
29          MOR
30      EVENING
31          EVE
32      MORNING
33      EVENING
34      MORNING
35    AFTERNOON
36      EVENING
37          MOR
38    AFTERNOON
39          AFT
40      EVENING
41          EVE
42      MORNING
43          AFT
44          MOR
45      EVENING
46          EVE
47      MORNING
48          MOR
49    AFTERNOON
Name: session, dtype: object

In [58]:
# Let use check out the counts of unique values inside the session Column
df.session.value_counts()

EVENING      10
MORNING       9
AFTERNOON     9
MOR           9
AFT           8
EVE           5
Name: session, dtype: int64

###  Handle  the Repeating Values under the session Column using `map()`
- To keep the data clean we will map all these values to only three categories to `MOR` , `AFT` and `EVE` using the map() function.
```
df.map(mapping, na_action=None)
```
- The `map()` method is used for substituting each value in a Series with another value, that may be derived from a `dict`. The `map()` method returns a series after performing the mapping
- You can give `ignore` as second argument which will propagate NaN values, without passing them to the mapping correspondence.

In [60]:
# To do this, let us create a new mapping (dictionary) 
dict1 = {
    'MORNING' : 'MOR',
    'MOR' : 'MOR',
    'AFTERNOON' : 'AFT',
    'AFT': 'AFT',
    'EVENING' : 'EVE',
    'EVE': 'EVE'
}

In [61]:
# It returns a series with the same index as caller, the original series remains unchanged. 
# So we have assigned the resulting series to `df.session` series
df.session.map(dict1)

0     MOR
1     AFT
2     EVE
3     MOR
4     AFT
5     AFT
6     EVE
7     EVE
8     MOR
9     AFT
10    MOR
11    MOR
12    MOR
13    AFT
14    AFT
15    EVE
16    MOR
17    AFT
18    AFT
19    MOR
20    AFT
21    EVE
22    MOR
23    AFT
24    AFT
25    EVE
26    EVE
27    MOR
28    AFT
29    MOR
30    EVE
31    EVE
32    MOR
33    EVE
34    MOR
35    AFT
36    EVE
37    MOR
38    AFT
39    AFT
40    EVE
41    EVE
42    MOR
43    AFT
44    MOR
45    EVE
46    EVE
47    MOR
48    MOR
49    AFT
Name: session, dtype: object

In [62]:
df.session = df.session.map(dict1)

In [63]:
# Count of new categories in the column session
# Observe we have managed to properly manage the values inside the session column
df.session.value_counts()

MOR    18
AFT    17
EVE    15
Name: session, dtype: int64

In [64]:
# Let us verify the result
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MOR,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFT,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVE,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFT,54,2100,76,78.0,55


## 7. Create a new Column by Modifying an Existing Column
- We have a column scholarship in the dataset, which is in Pak Rupees
- Suppose you want to have a new column which should represent the scholarship in US Dollars
- For that we need to add a new column by dividing each value of scholarship with 150

In [65]:
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv' )
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [66]:
df.scholarship.apply(lambda x: x/170)

0     15.070588
1     16.470588
2     20.588235
3     11.764706
4     12.352941
5     22.352941
6     17.647059
7     11.764706
8     14.705882
9     16.470588
10    17.647059
11    19.482353
12    13.794118
13    15.611765
14    12.570588
15    15.100000
16    20.588235
17    14.705882
18    17.647059
19    12.941176
20    20.588235
21    11.764706
22    14.705882
23    17.647059
24    14.705882
25    20.588235
26    14.705882
27    17.647059
28    23.529412
29    20.588235
30    14.705882
31    17.647059
32    20.588235
33    17.647059
34    14.705882
35    20.588235
36    17.647059
37    14.705882
38    20.588235
39    17.647059
40    14.705882
41    20.588235
42    17.647059
43    14.705882
44    20.588235
45    17.647059
46    14.705882
47    20.588235
48    14.705882
49    17.647059
Name: scholarship, dtype: float64

In [67]:
df['Scholarship_in_$'] = df.scholarship.apply(lambda x : x/150)

In [68]:
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu,Scholarship_in_$
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74,17.080000
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88,18.666667
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93,23.333333
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44,13.333333
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55,14.000000


## 8. Delete Rows Having NaN values using `df.dropna()` method

In [69]:
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv')
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [70]:
df.shape

(50, 10)

In [71]:
# You can use dropna() method to drop all the rows, it it has any na value
df1 = df.dropna()
df1.shape

(41, 10)

In [73]:
df1.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55
6,MS07,SARA,female,group B,EVENING,47,3000,88,95.0,92


In [ ]:
# Default Arguments to dropna()
df2 = df.dropna(axis=0, how='any')
df2.shape

In [74]:
# If we set how='all` it means drop a row only if all of its values are NA
df2 = df.dropna(axis=0, how='all')
df2.shape

(50, 10)

In [75]:
# Use of subset argument and pass it a list of columns based on whose values you want to drop a row
df2 = df.dropna(axis=0, how='any', subset=['math'])
df2.shape

(46, 10)

In [76]:
# Use of subset argument
df2 = df.dropna(axis=0, how='any', subset=['session'])
df2.shape

(50, 10)

In [78]:
# Having `how=all` and `subset=listofcolumnnames`, then it will 
# drop a row only if both the columns have a NA value in that row
df2 = df.dropna(axis=0, how='any', subset=['math', 'session'])
df2.shape

(46, 10)

In [79]:
# If we set the axis=1 and how=all, it means drop a column if all the  values under it is na
df2 = df.dropna(axis=1, how='all')
df2.shape

(50, 10)

In [80]:
# If we set the axis=1 and how=any, it means drop a column if any value under it is na
df2 = df.dropna(axis=1, how='any')
df2.shape

(50, 7)

In [81]:
df2.head()

,rollno,name,gender,session,age,scholarship,urdu
0,MS01,SAADIA,female,MORNING,28,2562,74
1,MS02,JUMAIMA,female,AFTERNOON,33,2800,88
2,MS03,ARIFA,female,EVENING,34,3500,93
3,MS04,SAADIA,female,MOR,44,2000,44
4,MS05,DANISH,male,AFTERNOON,54,2100,55


## 9. Convert Categorical Variables into Numerical
- Most of the machine learning algorithms do not take categorical variables so we need to convert them into numerical ones. 
- We can do this using Pandas function `pd.get_dummies()`, which will create a binary column for each of the categories. 
```
pd.get_dummies(data, drop_first=False)
```
- Where, the only required argument is `data` which can be a dataframe or a series
- The parameter drop_first : bool, default False Whether to get k-1 dummies out of k categorical levels by removing the first level.

**Note:** Making a dummy variable will take all the `K` distinct values in one coumn and make `K` columns out of them

### a. Convert all categorical variables into dummy/indicator variables

In [82]:
import pandas as pd
df = pd.read_csv('datasets/group-marks.csv')
df.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [83]:
# currently we have 10 columns in the data
df.shape

(50, 10)

In [84]:
# Convert all categorical variables into dummy/indicator variables
df = pd.get_dummies(df)

In [85]:
# Let us view the datafreame, keep a note on the number of columns
df.head()

,age,scholarship,english,urdu,rollno_MS01,rollno_MS02,rollno_MS03,rollno_MS04,rollno_MS05,rollno_MS06,...,math_70,math_73,math_74,math_75,math_76,math_78,math_81,math_88,math_97,math_No Idea
0,28,2562,72.0,74,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,33,2800,90.0,88,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34,3500,95.0,93,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44,2000,57.0,44,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,54,2100,78.0,55,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# The Number of columns has gone to 1605 now
df.shape

- So we have 112 columns
- Even though one-hot encoding is a good way to convert your categorical columns to numerical columns
- But it adds a lot of dimensionality to your data, i.e., increase the number of columns
- It also become difficult to deal with that much number of columns
- This is a trade-off, which is handled by technique called dimensionality reduction

### b. Perform One-Hot Encoding for Categorical Column `gender` Only
- In our dataframe, the gender column is a categorical column having two values 'male' and 'female'
- It will create a dummy binary columns.  
- This is also known as `One Hot Encoding`. You will learn more encoding techniques in the data pre-processing module.


In [86]:
import pandas as pd
df1 = pd.read_csv('datasets/group-marks.csv')
df1.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55


In [87]:
# Convert only gender variable into dummy/indicator variables
df2 = pd.get_dummies(df1[['gender']])
df2.head()

,gender_female,gender_male
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1


In [88]:
# Since we donot need two separate columns, so simply use the `drop_first` argument of get_dummies to handle this
df2 = pd.get_dummies(df1[['gender']], drop_first=True)
df2.head()

,gender_male
0,0
1,0
2,0
3,0
4,1


In [89]:
# We will talk about join in the next session in detail.
df3 = df1.join(df2['gender_male'])
df3.head()

,rollno,name,gender,group,session,age,scholarship,math,english,urdu,gender_male
0,MS01,SAADIA,female,group B,MORNING,28,2562,No Idea,72.0,74,0
1,MS02,JUMAIMA,female,group C,AFTERNOON,33,2800,69,90.0,88,0
2,MS03,ARIFA,female,NaN,EVENING,34,3500,NaN,95.0,93,0
3,MS04,SAADIA,female,group A,MOR,44,2000,47,57.0,44,0
4,MS05,DANISH,male,group C,AFTERNOON,54,2100,76,78.0,55,1
